In [19]:
#os.environ['OPENAI_API_KEY'] = <your-api-key>
import openai
import os

api_key = os.getenv('OPENAI_API_KEY')

https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

In [2]:

import pandas as pd
import re
def data_loader():
  """Loads data from CSV files, performs data cleaning and feature engineering.

  Returns:
    pandas.DataFrame: The processed hotel reviews dataset.
  """

  # Read hotel reviews data
  Hotel_Reviews = pd.read_csv("../Data/Hotel_Reviews.csv")

  # Read country data
  countries = pd.read_csv("../Data/countries.csv")

  # Add a "Country" column with NA values
  Hotel_Reviews["Country"] = pd.NA

  # Assign country based on pattern matching in Hotel_Address
  for country_name in countries["name"]:
    pattern = re.compile(country_name, re.IGNORECASE)  # Case-insensitive matching
    Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name

 
  # Convert Review_Date to datetime and extract features
  Hotel_Reviews["date_object"] = pd.to_datetime(Hotel_Reviews["Review_Date"], format="%m/%d/%Y")
  #Hotel_Reviews["time"] = Hotel_Reviews["date_object"].astype(int) / 10**9  # Convert to Unix timestamp
  Hotel_Reviews["month"] = Hotel_Reviews["date_object"].dt.month
  Hotel_Reviews["num_date_object"] = Hotel_Reviews["date_object"].dt.day_of_year / 365  # Normalize by days in a year

  return Hotel_Reviews

# Load the processed data
Hotel_Reviews = data_loader()

C:\Users\alan_\AppData\Local\Temp\ipykernel_26464\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_26464\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_26464\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_26464\278692378.py:22: UserWarning: This pattern is inte

#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter



# Merge the columns using string concatenation
Hotel_Reviews['MergedColumn_topic'] = (
    Hotel_Reviews['Positive_Review'] + 
    '. ' + Hotel_Reviews['Negative_Review'] 
)
# Select the first 100 rows of the merged column
Hotel_Reviews_UK =Hotel_Reviews[Hotel_Reviews["Country"]== "United Kingdom"]
used_data = Hotel_Reviews_UK['Positive_Review']#Hotel_Reviews['MergedColumn_topic']#[:2000]

# Specify the file name
file_name = "used_data.txt"

# Save the data to a text file
with open(file_name, 'w') as f:
    for line in used_data:
        f.write(line + '\n')

    

In [4]:
docs = used_data.tolist()


Define get_embeddings Function: This function takes a text input, tokenizes it, passes it through the BERT model, and returns the mean of the last hidden state as the text's embedding.
Define Topics and Keywords: It defines a dictionary of topics, each associated with a list of relevant keywords.
Example Texts: It assumes docs[:5] contains example texts to be classified.
Similarity Threshold: A threshold of 0.25 is set to determine if a text is similar enough to a topic.
Classify Texts: For each text:
It computes the embedding of the text.
For each topic, it computes the average embedding of its keywords.
It calculates the cosine similarity between the text's embedding and each topic's average keyword embedding.
It assigns the text to topics where the similarity exceeds the threshold.
Output: It prints the topics under which each text is classified or indicates if no topics match above the threshold

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Define your topics and keywords
topics = {
    "Room and Service": ["room", "rooms", "upgrade", "clean", "tidy", "smell", "large","bathroom", "bed", "beds", "TV", "bar", "conditioner","shower", "Service","friendly"],
    "Food and Drinks": ["drinks", "cocktails", "champagne", "bottle", "breakfast", "dinner", "menu", "caffee", "tee", "delicious", "continental", "waiter"],
    "Location": ["close", "far", "next", "park", "train", "bicicle", "car", "walk", "tee", "building", "neighborhood","cab service","airport", "underground","stairs"],
    "Internet and Work": ["wifi", "Internet", "connection", "work", "password", "computer", "meeting", "signal"],
    "Surprise": ["everything", "honestly", "surprising", "change", "unfortunately", "refund","loud"],
}



# Example texts
texts = docs[:10]

# Define a similarity threshold
similarity_threshold = 0.25

# Classify each text
for text in texts:
    text_embedding = get_embeddings(text)
    similarities = {}
    for topic, keywords in topics.items():
        keyword_embeddings = [get_embeddings(keyword) for keyword in keywords]
        avg_keyword_embedding = np.mean(keyword_embeddings, axis=0)
        similarity = cosine_similarity(text_embedding, avg_keyword_embedding.reshape(1, -1))
        similarities[topic] = similarity[0][0]

    # Assign topics with similarity above the threshold
    assigned_topics = [topic for topic, similarity in similarities.items() if similarity > similarity_threshold]
    print(similarities)
    if assigned_topics:
        print(f"The text '{text}' is classified under the topics: {', '.join(assigned_topics)}")
    else:
        print(f"The text '{text}' does not match any topic above the threshold.")

{'Room and Service': 0.36707085, 'Food and Drinks': 0.35206914, 'Location': 0.3383775, 'Internet and Work': 0.36096442, 'Surprise': 0.30324674}
The text ' Very comfortable beds smart bathroom good shower Lovely facilities garden an absolute delight friendly staff quiet location' is classified under the topics: Room and Service, Food and Drinks, Location, Internet and Work, Surprise
{'Room and Service': 0.7171439, 'Food and Drinks': 0.6949233, 'Location': 0.7138995, 'Internet and Work': 0.7179507, 'Surprise': 0.69527644}
The text 'No Positive' is classified under the topics: Room and Service, Food and Drinks, Location, Internet and Work, Surprise
{'Room and Service': 0.31756565, 'Food and Drinks': 0.3407111, 'Location': 0.3211431, 'Internet and Work': 0.3200249, 'Surprise': 0.2940263}
The text ' The staff was very friendly and helpful in organizing our daytime activities Hotel located in a quiet area but very close to subway and restaurants ' is classified under the topics: Room and Ser

{'Room': 0.28226188,
 'Food and Drinks': 0.2918899,
 'Location': 0.26071373,
 'Internet and Work': 0.26539245,
 'Surprise': 0.23990743}

In [13]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Use the [CLS] token representation for sentence embedding
    cls_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return cls_embedding

# Define your topics and keywords
topics = {
    "Room and Service": ["room", "rooms", "upgrade", "clean", "tidy", "smell", "large","bathroom", "bed", "beds", "TV", "bar", "conditioner","shower", "Service","friendly"],
    "Food and Drinks": ["drinks", "cocktails", "champagne", "bottle", "breakfast", "dinner", "menu", "caffee", "tee", "delicious", "continental", "waiter"],
    "Location": ["close", "far", "next", "park", "train", "bicicle", "car", "walk", "tee", "building", "neighborhood","cab service","airport", "underground","stairs"],
    "Internet and Work": ["wifi", "Internet", "connection", "work", "password", "computer", "meeting", "signal"],
    "Surprise": ["everything", "honestly", "surprising", "change", "unfortunately", "refund","loud"],
    #general
}


# Example texts
#texts = ["The room was clean and the bed was comfortable.", "The cocktails at the bar were delicious.", "The hotel is close to the airport and has a cab service.", "The wifi connection was strong and perfect for work.", "Honestly, the refund process was surprisingly smooth."]
texts = docs[:10]

# Define a similarity threshold
similarity_threshold = 0.5

# Classify each text
for text in texts:
    text_embedding = get_embeddings(text)
    similarities = {}
    for topic, keywords in topics.items():
        keyword_embeddings = [get_embeddings(keyword) for keyword in keywords]
        avg_keyword_embedding = np.mean(keyword_embeddings, axis=0)
        similarity = cosine_similarity(text_embedding, avg_keyword_embedding.reshape(1, -1))
        similarities[topic] = similarity[0][0]

    # Assign topics with similarity above the threshold
    assigned_topic = max(similarities, key=similarities.get)
    
    print(f"The text '{text}' is classified under the topic: {assigned_topic}")


The text ' Very comfortable beds smart bathroom good shower Lovely facilities garden an absolute delight friendly staff quiet location' is classified under the topic: Room and Service
The text 'No Positive' is classified under the topic: Room and Service
The text ' The staff was very friendly and helpful in organizing our daytime activities Hotel located in a quiet area but very close to subway and restaurants ' is classified under the topic: Location
The text ' We stayed at the hotel at a friend s recommendation It has great access to public transport and plenty of shops and places to eat close by Laundrette around the corner is also a bonus for long term travellers who need to freshen things up having access to the back garden was an added bonus for lovely summer evenings Would definitely stay there again ' is classified under the topic: Food and Drinks
The text ' Super location on Division underground line direct to Heathrow and central London Older facility but very accommodating s

In [5]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="your_app_name")
location = geolocator.geocode("Crete")
if location:
    print((location.latitude, location.longitude))

(41.4445334, -87.6313829)


In [10]:
pip install folium

  Using cached xyzservices-2024.9.0-py3-none-any.whl.metadata (4.1 kB)
Using cached xyzservices-2024.9.0-py3-none-any.whl (85 kB)
Note: you may need to restart the kernel to use updated packages.


In [32]:
import folium
from geopy.geocoders import Nominatim

# Function to get geolocation data
def get_geolocation(city_name):
    geolocator = Nominatim(user_agent="geo_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    return None, None

# Example city name
city_name = "Koum Kapi, Chania"

# Get geolocation data
latitude, longitude = get_geolocation(city_name)

# Check if geolocation data is available
if latitude is not None and longitude is not None:
    # Create a map centered around the location
    m = folium.Map(location=[latitude, longitude], zoom_start=12)

    # Add a marker for the location
    folium.Marker([latitude, longitude], popup=city_name).add_to(m)

    # Display the map
    m
else:
    print("Location not found.")

    

In [71]:
import folium
from geopy.geocoders import Nominatim

# Function to get geolocation data
def get_geolocation(description):
    geolocator = Nominatim(user_agent="geo_app")
    location = geolocator.geocode(description)
    if location:
        return location.latitude, location.longitude
    return None, None

# Function to try multiple geocoding strategies
def find_location(description):
    # Try the full description first
    latitude, longitude = get_geolocation(description)
    if latitude is not None and longitude is not None:
        return latitude, longitude
    
    # If not found, simplify the description and try again
    simplified_description = description.split(',')[-1].strip()
    return get_geolocation(simplified_description)

# Example location descriptions
location_descriptions = ["Chania Altstadt, Chania"]
for description in location_descriptions:
    # Find the location using multiple strategies
    latitude, longitude = find_location(description)
    
    # Check if geolocation data is available
    if latitude is not None and longitude is not None:
        # Create a map centered around the location
        m = folium.Map(location=[latitude, longitude], zoom_start=9)

        # Add a marker for the location
        folium.Marker([latitude, longitude], popup=description).add_to(m)

        # Display the map
        display(m)
    else:
        print(f"Location not found for: {description}")

In [79]:
description = "Stadtzentrum, Iraklio"

find_location(description)


<function str.strip(chars=None, /)>